In [76]:
import pandas as pd

In [77]:
data = pd.read_csv("./un-negated_clean_data.csv")

In [78]:
data

,Tokens,Tags,is_negative,sentence
0,I,O,False,0
1,am,O,False,0
2,looking,O,False,0
3,for,O,False,0
4,a,O,False,0
...,...,...,...,...
2345,30,B-SIZE,False,166
2346,inch,I-SIZE,False,166
2347,white,B-COLOUR,False,166
2348,desk,B-PRODUCT,False,166


In [79]:
data = data[["Tokens","Tags","sentence"]]

In [80]:
data

,Tokens,Tags,sentence
0,I,O,0
1,am,O,0
2,looking,O,0
3,for,O,0
4,a,O,0
...,...,...,...
2345,30,B-SIZE,166
2346,inch,I-SIZE,166
2347,white,B-COLOUR,166
2348,desk,B-PRODUCT,166


In [81]:
data['Sentence']= data[['sentence','Tokens','Tags']].groupby(['sentence'])['Tokens'].transform(lambda x: ' '.join(x))

In [82]:
data['Tags']= data[['sentence','Tokens','Tags']].groupby(['sentence'])['Tags'].transform(lambda x: ','.join(x))

In [83]:
data = data[['Sentence','Tags']]

In [84]:
data = data.drop_duplicates().reset_index(drop=True)

In [85]:
data

,Sentence,Tags
0,I am looking for a black gloss 33 inch firecla...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-SIZE,I-SIZE,B-P..."
1,Looking for pre workout Pump addict instead of...,"O,O,B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,O,..."
2,i need a 48 inch glass sliding goof and a show...,"O,O,O,B-SIZE,I-SIZE,B-MATERIAL,B-PRODUCT,I-PRO..."
3,"Hello, do any of your free standing tubs have ...","O,O,O,O,O,B-ATTRIBUTE,I-ATTRIBUTE,B-PRODUCT,O,..."
4,I'm looking for a 24 inch white mirror that is...,"O,O,O,O,B-SIZE,I-SIZE,B-COLOUR,B-PRODUCT,O,O,B..."
...,...,...
162,What rectangular shower units are available?,"O,B-SHAPE,B-PRODUCT,I-PRODUCT,O,O"
163,I am looking for a 35 inch bathroom sink count...,"O,O,O,O,O,B-SIZE,I-SIZE,B-PRODUCT,I-PRODUCT,I-..."
164,I'm looking f or a Black Matte bathtub double ...,"O,O,O,O,O,B-COLOUR,B-TEXTURE,B-PRODUCT,I-PRODU..."
165,Need a 27 inch frameless shower door,"O,O,B-SIZE,I-SIZE,B-ATTRIBUTE,B-PRODUCT,I-PRODUCT"


In [86]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)

In [87]:
def strsplit_tags(tags):
    sent_list = tags.split(",")
    return sent_list

In [88]:
def strsplit_sentence(sentence):
    sent_list = sentence.split(" ")
    return sent_list

In [89]:
train_sents = []

for index, row in train_dataset.iterrows():
    train_sents.append((strsplit_sentence(row['Sentence']),strsplit_tags(row['Tags'])))


In [90]:
dev_sents = []

for index, row in test_dataset.iterrows():
    dev_sents.append((strsplit_sentence(row['Sentence']),strsplit_tags(row['Tags'])))

In [91]:
# my code here
import nltk

# names_gazetteer = set(names.words())
# location_gazetteer = set()
# for location in gazetteers.words():
#     words = location.split()
#     for word in words:
#         if word[0].isupper() and len(word) > 3:
#             location_gazetteer.add(word)
# my code here

def get_pos(word):
    tag = nltk.pos_tag([word])
    return tag[0][1]

def is_number(string):
    return any(char.isdigit() for char in string)

def word2features(sentence, idx):
    word_features = {}
    word_features['word_lowercase'] = sentence[idx].lower()

    # your code here
    ## Features looking at the neighbouring words:
    
    if idx > 0:
        word_features["pre_word"] = sentence[idx -1].lower()
    else:
        word_features["pre_word"] = ""
    if idx < len(sentence) - 1:
        word_features["next_word"] = sentence[idx +1].lower()
    else:
        word_features["next_word"] = ""
        
    if idx > 1:
        word_features["pre2_word"] = sentence[idx -2].lower()
    else:
        word_features["pre2_word"] = ""
        
    if idx < len(sentence) - 2:
        word_features["next2_word"] = sentence[idx +2].lower()
    else:
        word_features["next2_word"] = ""
    ## Features loking at the word endings
    
    if len(sentence[idx])> 2:
        word_features["last2char"] = sentence[idx][-2:]
    else:
        word_features["last2char"] = sentence[idx]
    
    if len(sentence[idx])> 3:
        word_features["last3char"] = sentence[idx][-3:]
    else:
        word_features["last3char"] = sentence[idx]
        
    ## Features considering the shape of the word
    
    if sentence[idx].isupper():
        word_features["upper"] = True
    else:
        word_features["upper"] = False  
        
    if sentence[idx].islower():
        word_features["lower"] = True
    else:
        word_features["lower"] = False 
    
    word_features["length"] = len(sentence[idx])
    word_features["position"] = idx
    
    ## Gazetteer features:
    
#     if sentence[idx] in names_gazetteer:
#         word_features["names_gazetteer"] = True
#     else:
#         word_features["names_gazetteer"] = False
        
#     if sentence[idx] in location_gazetteer:
#         word_features["location_gazetteer"] = True
#     else:
#         word_features["location_gazetteer"] = False
    
    ## Extra Features:
    
    ## Is Number
    word_features["number"] = is_number(sentence[idx])
    
#     if is_number(sentence[idx]) == True:
#         word_features["num_length"] = len(sentence[idx])
#     else:
#         word_features["num_length"] = 0
    
    ##Is_Noun
    
    if get_pos(sentence[idx])== "NN":
        word_features["is_noun"] = True
    else:
        word_features["is_noun"] = False
    
    ## Percent Feature:
    
#     if "%" in sentence[idx]:
#         word_features["percent"] = True
#     else:
#         word_features["percent"] = False

#     ## Money Feature:
    
#     if sentence[idx].lower() in ["million","billion","trillion","dollars","yuan"]:
#         word_features["money"] = True
#     else:
#         word_features["money"] = False
        
#     ## Cardinal Feature:
        
#     if sentence[idx].lower() in ["one","two","three","four","five","six","seven","eight","nine","ten"]:
#         word_features["cardinal"] = True
#     else:
#         word_features["cardinal"] = False
        
#     if sentence[idx].lower() in ["year","day","week","days","years","weeks","date","centuries","century","january","february","march","april","may","june","july","august","september","october","november","december","monday","tuesday","wednesday","thursday","friday","saturday","sunday","today","tomorrow","yesterday"]:
#         word_features["date"] = True
#     else:
#         word_features["date"] = False
        
#     if sentence[idx].lower() in ["morning","afternoon","evening","night","tonight"]:
#         word_features["time"] = True
#     else:
#         word_features["time"] = False
        
    for i in range(len(sentence)):
        if i == idx and i != 0:
            word_features['first_word_not_in_title_case'] = sentence[idx].istitle()
        elif i == idx and i == 0:
            if sentence[idx].istitle():
                word_features['first_word_not_in_title_case'] = False
    
        
    return word_features
    
    
def sentence2features(sentence):
    return [word2features(sentence, idx) for idx in range(len(sentence))]

In [92]:
def prepare_ner_feature_dicts(sents):
    '''ner_files is a list of Ontonotes files with NER annotations. Returns feature dictionaries and 
    IOB tags for each token in the entire dataset'''
    all_dicts = []
    all_tags = []
    # your code here
    for tokens, tags in sents:
        all_dicts.extend(sentence2features(tokens))
        all_tags.extend(tags)
    
    return all_dicts, all_tags

In [93]:
train_dicts, train_tags = prepare_ner_feature_dicts(train_sents)
dev_dicts, dev_tags = prepare_ner_feature_dicts(dev_sents)

In [94]:
len(train_dicts)

1873

In [95]:
len(train_tags)

1873

In [96]:

import os
import nltk
nltk.download("gazetteers")
nltk.download("names")
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import f1_score, classification_report
from bs4 import BeautifulSoup
from nltk.corpus import names,gazetteers 
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report
import re

[nltk_data] Downloading package gazetteers to
[nltk_data]     /Users/varadrajrameshpoojary/nltk_data...
[nltk_data]   Package gazetteers is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     /Users/varadrajrameshpoojary/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [97]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer()
train_dicts = vec.fit_transform(train_dicts)
dev_dicts = vec.transform(dev_dicts)
clf = MultinomialNB()
clf.fit(train_dicts, train_tags)
y_pred = clf.predict(dev_dicts)

# your code here
print("MicroF1:",f1_score(dev_tags, y_pred,average="micro"))
print("MacroF1:",f1_score(dev_tags, y_pred,average="macro"))
print(classification_report(dev_tags, y_pred))

MicroF1: 0.6918238993710691
MacroF1: 0.17146205831151434
              precision    recall  f1-score   support

 B-ATTRIBUTE       0.38      0.10      0.16        30
    B-COLOUR       0.00      0.00      0.00        10
  B-MATERIAL       0.00      0.00      0.00         7
     B-PRICE       0.00      0.00      0.00         1
   B-PRODUCT       0.61      0.50      0.55        44
     B-SHAPE       0.00      0.00      0.00         1
      B-SIZE       1.00      0.33      0.50        21
   B-TEXTURE       0.00      0.00      0.00         2
 I-ATTRIBUTE       0.00      0.00      0.00        12
    I-COLOUR       0.00      0.00      0.00         3
  I-MATERIAL       0.00      0.00      0.00         1
   I-PRODUCT       0.50      0.03      0.06        34
      I-SIZE       1.00      0.19      0.32        16
           O       0.70      1.00      0.82       295

    accuracy                           0.69       477
   macro avg       0.30      0.15      0.17       477
weighted avg       0.63

/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### CRF

In [98]:
def prepare_ner_feature_dicts(sents):
    '''ner_files is a list of Ontonotes files with NER annotations. Returns feature dictionaries and 
    IOB tags for each token in the entire dataset'''
    all_dicts = []
    all_tags = []
    # your code here
    for tokens, tags in sents:
        all_dicts.append(sentence2features(tokens))
        all_tags.append(tags)

    return all_dicts, all_tags

In [99]:
train_dicts, train_tags = prepare_ner_feature_dicts(train_sents)
dev_dicts, dev_tags = prepare_ner_feature_dicts(dev_sents)

In [100]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=350,
    all_possible_transitions=True,
    verbose = True
)
crf.fit(train_dicts, train_tags)
try:
    call_produces_an_error()
except:
    pass

loading training data to CRFsuite: 100%|██████████| 134/134 [00:00<00:00, 7355.06it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 5436
Seconds required: 0.010

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 350
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.01  loss=4554.97  active=5372  feature_norm=0.50
Iter 2   time=0.00  loss=3851.87  active=5281  feature_norm=0.36
Iter 3   time=0.00  loss=3554.93  active=5257  feature_norm=0.28
Iter 4   time=0.00  loss=3489.67  active=5366  feature_norm=0.32
Iter 5   time=0.00  loss=3247.19  active=5315  feature_norm=0.66
Iter 6   time=0.00  loss=3117.22  active=5329  feature_norm=0.81
Iter 7   time=0.00  loss=3007.47  active=5363  feature_norm=1.00
Iter 8   time=0.00  loss=2786.27  active=5354  feature_norm=1.50
Iter 9   time=0.00  loss=2470.73  active=5373  feature_norm=2.79
Iter 10  time=0

In [101]:
def flatten(l):
    result = []
    for sub in l:
        result.extend(sub)
    return result

y_pred = crf.predict(dev_dicts)
print(f1_score(flatten(dev_tags), flatten(y_pred), average='macro'))
print(f1_score(flatten(dev_tags), flatten(y_pred), average='micro'))
print(classification_report(flatten(dev_tags), flatten(y_pred)))

0.4863159653955721
0.7777777777777778
              precision    recall  f1-score   support

 B-ATTRIBUTE       0.48      0.43      0.46        30
    B-COLOUR       1.00      0.20      0.33        10
  B-MATERIAL       0.67      0.29      0.40         7
     B-PRICE       0.00      0.00      0.00         1
   B-PRODUCT       0.55      0.50      0.52        44
     B-SHAPE       1.00      1.00      1.00         1
      B-SIZE       0.94      0.71      0.81        21
   B-TEXTURE       1.00      0.50      0.67         2
 I-ATTRIBUTE       0.33      0.42      0.37        12
    I-COLOUR       0.00      0.00      0.00         3
  I-MATERIAL       0.00      0.00      0.00         1
   I-PRODUCT       0.50      0.56      0.53        34
      I-SIZE       0.81      0.81      0.81        16
           O       0.87      0.94      0.91       295

    accuracy                           0.78       477
   macro avg       0.58      0.45      0.49       477
weighted avg       0.77      0.78      0.7

/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
